In [1]:
import os
import sys

import pandas as pd
import numpy as np

import scipy.stats as stats
import statsmodels.api as sm 
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [2]:
# Read in data
os.chdir("/Users/andreamendoza/Desktop/ECO 726")
os.getcwd()
os.path.dirname(os.getcwd())
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, 'data')
DATA_DIR
csv_file = os.path.join(DATA_DIR, 'bracero_aer_dataset_prep.csv')
df = pd.read_csv(csv_file)
df.shape

(15831, 105)

In [3]:
df.head()

,State,State_FIPS,Region,Year,Month,DailywoBoard_final,HourlyComposite_final,MonthlyComposite_final,TotalHiredSeasonal_final,Mexican_final,...,realwage_hourly,mex_frac_55,_merge,post,treatment_frac,none,domestic_seasonal,ln_domestic_seasonal,ln_realwage_hourly,ln_realwage_daily
0,AL,1,East South Central,1942,Jan,1.20,NaN,23.25,NaN,NaN,...,NaN,0.0,Matched (3),0,0.0,1,NaN,NaN,NaN,0.869079
1,AL,1,East South Central,1942,Apr,1.30,NaN,25.25,NaN,NaN,...,NaN,0.0,Matched (3),0,0.0,1,NaN,NaN,NaN,0.923963
2,AL,1,East South Central,1942,Jul,1.35,NaN,26.50,NaN,NaN,...,NaN,0.0,Matched (3),0,0.0,1,NaN,NaN,NaN,0.943241
3,AL,1,East South Central,1942,Oct,1.65,NaN,30.50,NaN,NaN,...,NaN,0.0,Matched (3),0,0.0,1,NaN,NaN,NaN,1.125784
4,AL,1,East South Central,1943,Jan,1.65,NaN,31.75,NaN,NaN,...,NaN,0.0,Matched (3),0,0.0,1,NaN,NaN,NaN,1.113880


In [4]:
 
# tab quarterly flag
quarterly_summary = df['quarterly_flag'].value_counts()
   
# sort by index 
quarterly_summary = quarterly_summary.sort_index()
   
# display the summary
print(quarterly_summary)

# keep if quarterly flag 
quarterly_data = df.loc[df['quarterly_flag'] == 1].copy()

# convert time_q from string to int for regression analysis 
def quarter_to_int(q):
    year = int(q[:4])
    quarter = int(q[-1])
    return 4 * (year - 1960) + (quarter - 1)

df["time_q_int"] = df["time_q"].apply(quarter_to_int)

# Get basic summary statistics
summary = df['time_q_int'].describe()
print("Summary of time_q_int:")
print(summary)
   

quarterly_flag
0    9447
1    6384
Name: count, dtype: int64
Summary of time_q_int:
count    15831.000000
mean        -1.896027
std         37.275377
min        -72.000000
25%        -31.000000
50%          2.000000
75%         30.000000
max         60.000000
Name: time_q_int, dtype: float64


In [5]:
# make sure nothing else is already in index, eststo clear
df = df.reset_index(drop=True)

In [6]:
def get_significance_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.10:
        return "*"
    else:
        return ""

# store treatment_frac regression estimates
def extract_tf(res):
    # Convert numpy arrays to Series with variable names
    params = pd.Series(res.params, index=res.model.exog_names)
    tvals  = pd.Series(res.tvalues, index=res.model.exog_names)
    pvals  = pd.Series(res.pvalues, index=res.model.exog_names)

    coef = params["treatment_frac"]
    tval = tvals["treatment_frac"]
    pval = pvals["treatment_frac"]

    stars = get_significance_stars(pval)

    return {
        "coef": f"{coef:.3f}{stars}",
        "t": f"({tval:.2f})",
        "N": int(res.nobs),
    }

In [7]:
# hourly wage, all years
df_clean_h = df.dropna(subset=['realwage_hourly', 'treatment_frac', 'State_FIPS', 'time_q_int'])
formula = "realwage_hourly ~ treatment_frac + C(State_FIPS) + C(time_q_int)"
res1 = smf.ols(formula, data=df_clean_h).fit()
res = res1.get_robustcov_results(cov_type='cluster', groups=df_clean_h['State_FIPS'])

# daily wage, all years
df_clean_d = df.dropna(subset=['realwage_daily', 'treatment_frac', 'State_FIPS', 'time_q_int'])
formula = "realwage_daily ~ treatment_frac + C(State_FIPS) + C(time_q_int)"
res2 = smf.ols(formula, data=df_clean_d).fit()
res = res2.get_robustcov_results(cov_type='cluster', groups=df_clean_d['State_FIPS'])

# hourly wage, 1960-1970 
df_clean_h = df.dropna(subset=['realwage_hourly', 'treatment_frac', 'State_FIPS', 'time_q_int'])
df_60_70 = df_clean_h[(df_clean_h['Year'] >= 1960) & (df_clean_h['Year'] <= 1970)]
formula = "realwage_hourly ~ treatment_frac + C(State_FIPS) + C(time_q_int)"
res3 = smf.ols(formula, data=df_60_70).fit()
res = res3.get_robustcov_results(cov_type='cluster', groups=df_60_70['State_FIPS'])

# daily wage, 1960-1970 
df_clean_d = df.dropna(subset=['realwage_daily', 'treatment_frac', 'State_FIPS', 'time_q_int'])
df_60_70_d = df_clean_d[(df_clean_d['Year'] >= 1960) & (df_clean_d['Year'] <= 1970)]
formula = "realwage_daily ~ treatment_frac + C(State_FIPS) + C(time_q_int)"
res4 = smf.ols(formula, data=df_60_70_d).fit()
res = res4.get_robustcov_results(cov_type='cluster', groups=df_60_70_d['State_FIPS'])



In [8]:
# Generate a table for regression output

out1 = extract_tf(res1)
out2 = extract_tf(res2)
out3 = extract_tf(res3)
out4 = extract_tf(res4)

from tabulate import tabulate
results = {
    "Hourly, all years": extract_tf(res1),
    "Daily, all years": extract_tf(res2),
    "Hourly, 1960-1970": extract_tf(res3),
    "Daily, 1960-1970": extract_tf(res4),
}
print("Effects of Bracero Exclusion on Daily and Hourly Wages, Quarterly Analysis\n")
df_table = pd.DataFrame(results)
print(tabulate(df_table, headers="keys", tablefmt="github"))

Effects of Bracero Exclusion on Daily and Hourly Wages, Quarterly Analysis

|      | Hourly, all years   | Daily, all years   | Hourly, 1960-1970   | Daily, 1960-1970   |
|------|---------------------|--------------------|---------------------|--------------------|
| coef | -0.036**            | -0.385***          | -0.040*             | -0.025             |
| t    | (-2.12)             | (-2.80)            | (-1.86)             | (-0.16)            |
| N    | 4324                | 5813               | 2024                | 1901               |
